# Import Packages

In [1]:
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from statsmodels.stats.weightstats import ttest_ind
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from datetime import datetime
from sklearn.utils import resample
from random import choices
from statsmodels.stats.multitest import multipletests

In [2]:
try:
  from stargazer.stargazer import Stargazer
except ImportError:
  %pip install stargazer
  from stargazer.stargazer import Stargazer

# Import data

In [3]:
# import data
baseline = pd.read_stata('data/baseline.dta')
cleanpricedata_y1y2 = pd.read_stata('data/cleanPriceData_Y1Y2.dta')
ms1ms2_pooled = pd.read_stata('data/MS1MS2_pooled.dta')

# this data is not needed for our analysis
# bok_inflation = pd.read_stata('data/BOK_inflation.dta')
# intensity_obs_short = pd.read_stata('data/intensity_obs_short.dta')
# lrfu_select_dataset = pd.read_stata('data/LRFU_select_dataset.dta')
# repayment_datay1 = pd.read_stata('data/repayment_dataY1.dta')

# Recreating the tables from the paper

## Table 1

We start by cleaning the data

In [4]:
# clean ms1ms2_pooled (drop if MS !=2, keep columns oafid and treatMS1MS2, group by oafid and take mean and rename)
ms1ms2_pooled_tab1 = ms1ms2_pooled[ms1ms2_pooled['MS']==2]
ms1ms2_pooled_tab1 = ms1ms2_pooled_tab1[['oafid', 'treatMS1MS2']]
ms1ms2_pooled_tab1 = ms1ms2_pooled_tab1.groupby('oafid', as_index=False).mean()
ms1ms2_pooled_tab1.rename(columns={'treatMS1MS2': 'treat13'}, inplace=True)
print(ms1ms2_pooled_tab1.shape[0]) # checking we have the right number of observations as described in the original article

1019


In [5]:
# clean baseline data (the stata code indicates that the variables columns 'businessprofitmonth' and 'delta' should be kept, however they have already been renamed to 'businessprofitmonth_base' and 'delta_base')
base_cols = ['oafid', 'logtotcons_base', 'male', 'num_adults', 'num_schoolchildren', 'finished_primary',
                   'finished_secondary', 'cropland', 'num_rooms', 'schoolfees', 'totcons_base', 'logpercapcons_base',
                   'total_cash_savings_base', 'total_cash_savings_trimmed', 'has_savings_acct', 'taken_bank_loan',
                   'taken_informal_loan', 'liquidWealth', 'wagepay', 'businessprofitmonth_base', 'price_avg_diff_pct',
                   'price_expect_diff_pct', 'harvest2011', 'netrevenue2011', 'netseller2011', 'autarkic2011',
                   'maizelostpct2011', 'harvest2012', 'correct_interest', 'digit_recall', 'maizegiver', 'delta_base', 'treatment']
baseline_clean = baseline[base_cols].copy()

# rename columns
baseline_clean.columns = [col + '_base' if not col.endswith('_base') and col != 'oafid' and col != 'treatment' else col for col in baseline_clean.columns]
baseline_clean.rename(columns={'treatment': 'treatment2012'}, inplace=True)

# generate treat12 as bool for treatment and control in 2012
baseline_clean['treat12'] = baseline_clean['treatment2012'].apply(lambda x: x in ['T1', 'T2'])
baseline_clean.loc[baseline_clean['treatment2012'] == '', 'treat12'] = np.nan

/var/folders/yw/jsw5n53s1cb1s2q6tt0msrm00000gn/T/ipykernel_54757/2284489521.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  baseline_clean.loc[baseline_clean['treatment2012'] == '', 'treat12'] = np.nan


In [6]:
# merge baseline_clean and ms1ms2_pooled_clean on oafid
base_ms1ms2_pool = pd.merge(baseline_clean, ms1ms2_pooled_tab1, on='oafid', how='left')

In [7]:
# create table 1
# copy in case we need this later
df_tab1 = base_ms1ms2_pool.copy()
df_tab1['schoolfees_base'] = df_tab1['schoolfees_base']*1000

# var list for table 1
vars_list = [
    "male_base", "num_adults_base", "num_schoolchildren_base", "finished_primary_base",
    "finished_secondary_base", "cropland_base", "num_rooms_base", "schoolfees_base",
    "totcons_base", "logpercapcons_base", "total_cash_savings_base",
    "total_cash_savings_trimmed_base", "has_savings_acct_base", "taken_bank_loan_base",
    "taken_informal_loan_base", "liquidWealth_base", "wagepay_base",
    "businessprofitmonth_base", "price_avg_diff_pct_base",
    "price_expect_diff_pct_base", "harvest2011_base", "netrevenue2011_base",
    "netseller2011_base", "autarkic2011_base", "maizelostpct2011_base",
    "harvest2012_base", "correct_interest_base", "digit_recall_base",
    "maizegiver_base"
]

renaming = {
    "male_base": "Male",
    "num_adults_base": "Number of adults",
    "num_schoolchildren_base": "Children in school",
    "finished_primary_base": "Finished primary school",
    "finished_secondary_base": "Finished secondary school",
    "cropland_base": "Total cropland (acres)",
    "num_rooms_base": "Number of rooms in household",
    "schoolfees_base": "Total school fees",
    "totcons_base": "Average monthly consumption (Ksh)",
    "logpercapcons_base": "Average monthly consumption/capita (log)",
    "total_cash_savings_base": "Total cash savings (Ksh)",
    "total_cash_savings_trimmed_base": "Total cash savings (trim)",
    "has_savings_acct_base": "Has bank savings acct",
    "taken_bank_loan_base": "Taken bank loan",
    "taken_informal_loan_base": "Taken informal loan",
    "liquidWealth_base": "Liquid wealth (Ksh)",
    "wagepay_base": "Off-farm wages (Ksh)",
    "businessprofitmonth_base": "Business profit (Ksh)",
    "price_avg_diff_pct_base": "Avg $\%\Delta$ price Sep-Jun",
    "price_expect_diff_pct_base": "Expect $\%\Delta$ price Sep12-Jun13",
    "harvest2011_base": "2011 LR harvest (bags)",
    "netrevenue2011_base": "Net revenue 2011 (Ksh)",
    "netseller2011_base": "Net seller 2011",
    "autarkic2011_base": "Autarkic 2011",
    "maizelostpct2011_base": "\% maize lost 2011",
    "harvest2012_base": "2012 LR harvest (bags)",
    "correct_interest_base": "Calculated interest correctly",
    "digit_recall_base": "Digit span recall",
    "maizegiver_base": "Maize giver"
}

# function to perform t-tests
def t_test_by_group(df, var, group_var='treat12'):
    group1 = df[df[group_var] == 0][var].dropna()
    group2 = df[df[group_var] == 1][var].dropna()
    t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=False)
    return group1.mean(), group2.mean(), len(group1) + len(group2), t_stat, p_val

# applying t-tests and collecting results
results = []
for var in vars_list:
    control_mean, treat_mean, obs, t_stat, p_val = t_test_by_group(df_tab1, var)
    std_diff = (treat_mean - control_mean) / np.sqrt(((len(df_tab1[df_tab1['treat12'] == 0][var]) - 1) * np.std(df_tab1[df_tab1['treat12'] == 0][var], ddof=1) ** 2 + (len(df_tab1[df_tab1['treat12'] == 1][var]) - 1) * np.std(df_tab1[df_tab1['treat12'] == 1][var], ddof=1) ** 2) / (len(df_tab1[df_tab1['treat12'] == 0][var]) + len(df_tab1[df_tab1['treat12'] == 1][var]) - 2))
    results.append([var, treat_mean, control_mean, obs, std_diff, p_val])

# convert results to a df to use pandas output to latex
results_df = pd.DataFrame(results, columns=['Variable', 'Treat Mean', 'Control Mean', 'Observations', 'Std Diff', 'P-value'])
results_df['Variable'] = results_df['Variable'].map(renaming)
results_df = results_df.rename(columns={
    'Variable':'Baseline characteristic', 
    'Treat Mean':'Treat', 
    'Control Mean':'Control', 
    'Observations':'Obs', 
    'Std Diff':'Std diff', 
    'P-value':'P-val'})

latex_table1 = results_df.to_latex(index=False, float_format="%.3f")
print(latex_table1)

\begin{tabular}{lrrrrr}
\toprule
Baseline characteristic & Treat & Control & Obs & Std diff & P-val \\
\midrule
Male & 0.296 & 0.334 & 1589 & -0.083 & 0.109 \\
Number of adults & 3.004 & 3.196 & 1510 & -0.099 & 0.067 \\
Children in school & 2.998 & 3.072 & 1589 & -0.038 & 0.454 \\
Finished primary school & 0.718 & 0.772 & 1490 & -0.122 & 0.019 \\
Finished secondary school & 0.253 & 0.270 & 1490 & -0.039 & 0.460 \\
Total cropland (acres) & 2.441 & 2.398 & 1512 & 0.014 & 0.796 \\
Number of rooms in household & 3.073 & 3.252 & 1511 & -0.072 & 0.219 \\
Total school fees & 27239.693 & 29813.631 & 1589 & -0.068 & 0.191 \\
Average monthly consumption (Ksh) & 14970.862 & 15371.378 & 1437 & -0.032 & 0.550 \\
Average monthly consumption/capita (log) & 7.975 & 7.963 & 1434 & 0.019 & 0.721 \\
Total cash savings (Ksh) & 5157.396 & 8021.499 & 1572 & -0.128 & 0.028 \\
Total cash savings (trim) & 4731.623 & 5389.836 & 1572 & -0.050 & 0.343 \\
Has bank savings acct & 0.419 & 0.425 & 1589 & -0.012 & 0.8

## Running the model for tables 2 through 4

In [8]:
treatments = ['treat12', 'treat13', 'treatMS1MS2']
dependent_vars = ['inventory_trim', 'netrevenue_trim','logtotcons_trim']

mean_df = pd.DataFrame()
std_df = pd.DataFrame()

results = {'netsales': {'overall': None, 'by_round':None}}

for dv in dependent_vars:
    for treat in treatments:
        # create df for each treatment
        if treat == 'treatMS1MS2':
            df1 = ms1ms2_pooled.loc[:, [dv,'treat12', 'Y1round1', 'Y1round2', 'Y1round3', 'treatMS1MS2', 'interviewdate', 'groupnum', 'strata_group']].copy(deep=True).dropna()
            df2 = ms1ms2_pooled.loc[:, [dv,'treat13', 'Y2round1', 'Y2round2', 'Y2round3', 'treatMS1MS2', 'interviewdate', 'groupnum', 'strata_group']].copy(deep=True).dropna()
            df1['inter_R1'] = df1['Y1round1'] * df1[f'treat12']
            df1['inter_R2'] = df1['Y1round2'] * df1[f'treat12']
            df1['inter_R3'] = df1['Y1round3'] * df1[f'treat12']
            df2['inter_R1'] = df2['Y2round1'] * df2[f'treat13']
            df2['inter_R2'] = df2['Y2round2'] * df2[f'treat13']
            df2['inter_R3'] = df2['Y2round3'] * df2[f'treat13']
            df = pd.concat([df1, df2], ignore_index=True).fillna(0)

            # model specification by round
            formula_by_round = f'{dv} ~ inter_R1 + inter_R2 + inter_R3 + interviewdate + C(Y1round1) + C(Y1round2) + C(Y1round3) + C(Y2round1) + C(Y2round2) + C(Y2round3) + C(strata_group)'
        else:
            if treat == 'treat12':
                year = 1
            else:
                year = 2
            df = ms1ms2_pooled.loc[:, [dv,treat, f'Y{year}round1', f'Y{year}round2', f'Y{year}round3', 'treatMS1MS2', 'interviewdate', 'groupnum', 'strata_group']].copy(deep=True).dropna()
            df['inter_R1'] = df[f'Y{year}round1'] * df[f'{treat}']
            df['inter_R2'] = df[f'Y{year}round2'] * df[f'{treat}']
            df['inter_R3'] = df[f'Y{year}round3'] * df[f'{treat}']


            # model specification by round
            formula_by_round = f'{dv} ~ inter_R1 + inter_R2 + inter_R3 + interviewdate + C(Y{year}round1) + C(Y{year}round2) + C(Y{year}round3) + C(strata_group)'
        
        df['z'] = df[treat]

        # specify overall model
        formula_overall = f'{dv} ~ z + interviewdate + C(strata_group)'

        # fit models
        model_overall = smf.ols(formula_overall, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['groupnum']})
        model_by_round = smf.ols(formula_by_round, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['groupnum']})

        # store models in dictionary
        results[f'{treat}_{dv}'] = {f'overall': model_overall, f'by_round': model_by_round}

        # extract necessary statistics
        mean_df.loc[dv, treat] = df[dv].mean()
        std_df.loc[dv, treat] = df[dv].std()


## Adding table 5

### Clean the data

In [9]:
ms1ms2_pooled_tab5 = ms1ms2_pooled.copy(deep=True)
max_strata_group = ms1ms2_pooled_tab5['strata_group'].max()
ms1ms2_pooled_tab5.loc[ms1ms2_pooled_tab5['MS'] == 2, 'strata_group'] = ms1ms2_pooled_tab5['groupstrata'] + max_strata_group

ms1ms2_pooled_tab5.loc[ms1ms2_pooled_tab5['MS'] == 2, 'oafid'] = ms1ms2_pooled_tab5['fr_id']

ms1ms2_pooled_tab5['purchasequant2'] = ms1ms2_pooled_tab5['purchasequant']
ms1ms2_pooled_tab5.loc[(ms1ms2_pooled_tab5['purchaseval']==0)&(ms1ms2_pooled_tab5['purchasequant'].isna()),'purchasequant2'] = 0
ms1ms2_pooled_tab5['netsales'] = ms1ms2_pooled_tab5['salesquant'] - ms1ms2_pooled_tab5['purchasequant2']

ms1ms2_pooled_tab5.drop(columns=['netsales_trim','purchaseval_trim','salesval_trim'], inplace=True)

In [10]:
# trim outliers
for x in ['purchaseval', 'salesval', 'purchasequant', 'salesquant']:
    quantile = np.quantile(ms1ms2_pooled_tab5[ms1ms2_pooled_tab5[x].notna()][x],[0.99],method='closest_observation')
    ms1ms2_pooled_tab5[f'{x}_trim'] = ms1ms2_pooled_tab5[x]
    ms1ms2_pooled_tab5.loc[ms1ms2_pooled_tab5[f'{x}_trim'] > quantile[0],f'{x}_trim'] = np.nan

quantile = np.quantile(ms1ms2_pooled_tab5[ms1ms2_pooled_tab5['netsales'].notna()]['netsales'],[0.005, 0.995],method='closest_observation')
ms1ms2_pooled_tab5['netsales_trim'] = ms1ms2_pooled_tab5['netsales']
ms1ms2_pooled_tab5.loc[(ms1ms2_pooled_tab5['netsales_trim'] <= quantile[0]) | (ms1ms2_pooled_tab5['netsales_trim'] > quantile[1]) , 'netsales_trim'] = np.nan

# create id
ms1ms2_pooled_tab5['id'] = ms1ms2_pooled_tab5['oafid'].fillna(ms1ms2_pooled_tab5['fr_id'])

# create effective prices
trim_vars = ['salesquant_trim', 'purchasequant_trim', 'salesval_trim', 'purchaseval_trim']
for var in trim_vars:
    ms1ms2_pooled_tab5[f'tot_{var}'] = ms1ms2_pooled_tab5.groupby(['id', 'MS'])[var].transform('sum')

for x in ['purchase', 'sales']:
    ms1ms2_pooled_tab5[f'effective_{x}_price'] = ms1ms2_pooled_tab5[f'tot_{x}val_trim'] / ms1ms2_pooled_tab5[f'tot_{x}quant_trim']
    ms1ms2_pooled_tab5.loc[ms1ms2_pooled_tab5[f'tot_{x}quant_trim']== 0,f'effective_{x}_price'] = np.nan

### Net sales

In [11]:
# define variable
dv = 'netsales_trim'
independent_vars = ['z', 'treatMS1MS2_1 + treatMS1MS2_2 + treatMS1MS2_3']

for i, var in enumerate(independent_vars):
    df = ms1ms2_pooled_tab5.copy(deep=True)
    df['z'] = df['treatMS1MS2']
    if var == 'z':
        df.dropna(subset=[dv,'z','interviewdate','Y1round2','Y1round3','Y2round1','Y2round2','Y2round3','strata_group','groupnum'], inplace=True)
    else:
        df.dropna(subset=[dv,'treatMS1MS2_1','treatMS1MS2_2','treatMS1MS2_3','interviewdate','Y1round2','Y1round3','Y2round1','Y2round2','Y2round3','strata_group','groupnum'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    formula = f'{dv} ~ {var} + interviewdate + Y1round2 + Y1round3 + Y2round1 + Y2round2 + Y2round3 + C(strata_group)'
    model = smf.ols(formula, df).fit(cov_type='cluster', cov_kwds={'groups': df['groupnum']})
    if i == 0:
        results['netsales']['overall'] = model
    else:
        results['netsales']['by_round'] = model
    
    mean_df.loc[dv, treat] = df.loc[df['treatMS1MS2'] == 0, dv].mean()
    std_df.loc[dv, treat] = df.loc[df['treatMS1MS2'] == 0, dv].std()


### Effective Price

In [12]:

for dv in ['purchase', 'sales']:
    for i, treat in enumerate(['treat12', 'treat13', 'treatMS1MS2']):
        df = ms1ms2_pooled_tab5.copy(deep=True)
        df['z'] = df[treat]
        df = df.drop_duplicates(subset=['id', 'MS'], keep='first')
        df.dropna(subset=[f'effective_{dv}_price','z','groupnum'], inplace=True)
        if treat == 'treatMS1MS2':
            formula = f'effective_{dv}_price ~ z + C(strata_group)'
        else:
            df = df[df['MS'] == i+1]
            formula = f'effective_{dv}_price ~ z + C(strata_group)'
        model = smf.ols(formula, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['groupnum']})
        results[f'{treat}_{dv}'] = {'overall':model}
        
        mean_df.loc[dv, treat] = df.loc[df['z'] == 0, f'effective_{dv}_price'].mean()
        std_df.loc[dv, treat] = df.loc[df['z'] == 0, f'effective_{dv}_price'].std()

In [13]:
mean_df

,treat12,treat13,treatMS1MS2
inventory_trim,3.021060,1.959206,2.559983
netrevenue_trim,485.811951,-2997.861572,-1033.442383
logtotcons_trim,9.476815,9.652856,9.553755
netsales_trim,NaN,NaN,-0.420827
purchase,2913.780762,3221.890625,3085.372070
sales,2774.760984,2858.969741,2809.763419


In [14]:
std_df

,treat12,treat13,treatMS1MS2
inventory_trim,3.725840,3.089480,3.503475
netrevenue_trim,6212.781250,6545.625977,6590.086426
logtotcons_trim,0.620554,0.651713,0.640296
netsales_trim,NaN,NaN,2.038441
purchase,481.983246,535.381042,534.511292
sales,503.057065,504.403012,504.821819


## Calculating FWER and pvals

In [15]:
pval_df = pd.DataFrame()

for treat in ['treat12', 'treat13', 'treatMS1MS2']:
    for dv in ['purchase', 'sales', 'inventory_trim', 'netrevenue_trim', 'logtotcons_trim']:
        pval = results[f'{treat}_{dv}']['overall'].pvalues['z']
        # store p-values in dataframe
        pval_df.loc[dv, treat] = pval
        
    fwer_pvals = multipletests(pval_df[treat], alpha=0.05, method='fdr_bh')[1]
    for i, dv in enumerate(['purchase', 'sales', 'inventory_trim', 'netrevenue_trim', 'logtotcons_trim']):
        pval_df.loc[dv, f'{treat}_fwer'] = fwer_pvals[i]


In [16]:
# index = pd.MultiIndex.from_product([['inventory_trim', 'netrevenue_trim', 'logtotcons_trim'], ['inter_R1', 'inter_R2', 'inter_R3']], names=['dv', 'var'])
pval_rd_df = pd.DataFrame()

for treat in ['treat12', 'treat13', 'treatMS1MS2']:
    for var in ['inter_R1', 'inter_R2', 'inter_R3']:
        for dv in ['inventory_trim', 'netrevenue_trim', 'logtotcons_trim']:
            pval = results[f'{treat}_{dv}']['by_round'].pvalues[var]
            # store p-values in dataframe
            pval_rd_df.loc[f'{dv}_{var}', f'{treat}_rd'] = pval

    fwer_pvals = multipletests(pval_rd_df[f'{treat}_rd'], alpha=0.05, method='fdr_bh')[1]
    for i, indx in enumerate(pval_rd_df.index):
        pval_rd_df.loc[indx, f'{treat}_fwer_rd'] = fwer_pvals[i]

In [17]:
# combine the p-values and split into two dfs
pvals = pd.concat([pval_df, pval_rd_df], axis=0)
pvals = pvals.map(lambda x: '<0.001' if x < 0.0005 else np.round(x,3))
pval_fwer = pvals[['treat12_fwer','treat12_fwer_rd','treat13_fwer','treat13_fwer_rd','treatMS1MS2_fwer','treatMS1MS2_fwer_rd']]
pval = pvals[['treat12','treat12_rd','treat13','treat13_rd','treatMS1MS2','treatMS1MS2_rd']]

## Outputting the tables

Creating stagazar for table 2-4

In [18]:
latex_tables = []
for i, dv in enumerate(['inventory_trim', 'netrevenue_trim', 'logtotcons_trim']):
    tables = []
    for treat in ['treat12', 'treat13', 'treatMS1MS2']:
        overall = results[f'{treat}_{dv}']['overall']
        by_rd = results[f'{treat}_{dv}']['by_round']
        tables.append(overall)
        tables.append(by_rd)
    stargazer = Stargazer(tables)
    stargazer.custom_columns(['Y1', 'Y2','Pooled'], [2,2,2])
    stargazer.significant_digits(3)
    stargazer.rename_covariates({'z': 'Treat', 'inter_R1': 'Treat - R1', 'inter_R2': 'Treat - R2', 'inter_R3': 'Treat - R3'})
    stargazer.covariate_order(['z', 'inter_R1', 'inter_R2', 'inter_R3'])
    
    # adding p-values
    stargazer.add_line('P-Val Treat', pval.loc[dv].tolist())
    stargazer.add_line('P-Val Treat FWER', pval_fwer.loc[dv].tolist())
    stargazer.add_line('P-Val Treat - R1', pval.loc[f'{dv}_inter_R1'].tolist())
    stargazer.add_line('P-Val Treat - R1 FWER', pval_fwer.loc[f'{dv}_inter_R1'].tolist())
    stargazer.add_line('P-Val Treat - R2', pval.loc[f'{dv}_inter_R2'].tolist())
    stargazer.add_line('P-Val Treat - R2 FWER', pval_fwer.loc[f'{dv}_inter_R2'].tolist())
    stargazer.add_line('P-Val Treat - R3', pval.loc[f'{dv}_inter_R3'].tolist())
    stargazer.add_line('P-Val Treat - R3 FWER', pval_fwer.loc[f'{dv}_inter_R3'].tolist())
    
    latex_table = stargazer.render_latex()

    # general formatting
    latex_table = latex_table.replace("Adjusted $R^2$", "% Adjusted $R^2$")
    latex_table = latex_table.replace("Residual Std. Error", "% Residual Std. Error")
    latex_table = latex_table.replace("F Statistic", "% F Statistic")
    latex_table = latex_table.replace("\\textit{Note","% \\textit{Note")
    latex_table = latex_table.replace("nan","")
    latex_table = latex_table.replace("\\begin{table}[!htbp] \\centering", "")
    latex_table = latex_table.replace("\\end{table}", "")

    # renaming variables
    latex_table = latex_table.replace("\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\",
                                "\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \n \\\ & Overall & By rd & Overall & By rd & Overall & By rd \\")
    latex_table = latex_table.replace("netsales_trim","Net Sales Trim")
    latex_table = latex_table.replace("inventory_trim","Inventory Trim")
    latex_table = latex_table.replace("logtotcons_trim","Log Total HH Consumption Trim")

    latex_tables.append(latex_table)

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 37, but rank is 36
  warnings.warn('covariance of constraints does not have full '
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 37, but rank is 36
  warnings.warn('covariance of constraints does not have full '
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 63, but rank is 62
  warnings.warn('covariance of constraints does not have full '
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 71, but rank is 68
  warnings.warn('covariance of constraints does not 

### Table 2

In [19]:
print(latex_tables[0])


\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable: Inventory Trim}} \
\cr \cline{2-7}
\\[-1.8ex] & \multicolumn{2}{c}{Y1} & \multicolumn{2}{c}{Y2} & \multicolumn{2}{c}{Pooled}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) 
 \\ & Overall & By rd & Overall & By rd & Overall & By rd \\
\hline \\[-1.8ex]
 Treat & 0.574$^{***}$ & & 0.546$^{***}$ & & 0.565$^{***}$ & \\
& (0.140) & & (0.129) & & (0.097) & \\
 Treat - R1 & & 0.872$^{***}$ & & 1.242$^{***}$ & & 1.050$^{***}$ \\
& & (0.276) & & (0.235) & & (0.184) \\
 Treat - R2 & & 0.753$^{***}$ & & 0.304$^{*}$ & & 0.546$^{***}$ \\
& & (0.171) & & (0.166) & & (0.120) \\
 Treat - R3 & & 0.111$^{}$ & & 0.082$^{}$ & & 0.094$^{}$ \\
& & (0.083) & & (0.344) & & (0.162) \\
 P-Val Treat & <0.001 &  & <0.001 &  & <0.001 &  \\
 P-Val Treat FWER & <0.001 &  & <0.001 &  & <0.001 &  \\
 P-Val Treat - R1 &  & 0.002 &  & <0.001 &  & <0.001 \\
 P-Val Treat - R1 FWER &  & 0.004

### Table 3

In [20]:
print(latex_tables[1])


\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable: netrevenue_trim}} \
\cr \cline{2-7}
\\[-1.8ex] & \multicolumn{2}{c}{Y1} & \multicolumn{2}{c}{Y2} & \multicolumn{2}{c}{Pooled}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) 
 \\ & Overall & By rd & Overall & By rd & Overall & By rd \\
\hline \\[-1.8ex]
 Treat & 263.790$^{}$ & & 854.114$^{***}$ & & 531.358$^{***}$ & \\
& (255.661) & & (303.802) & & (196.315) & \\
 Treat - R1 & & -1164.574$^{***}$ & & 16.478$^{}$ & & -613.581$^{**}$ \\
& & (322.956) & & (444.957) & & (271.653) \\
 Treat - R2 & & 509.851$^{}$ & & 1994.923$^{***}$ & & 1187.967$^{***}$ \\
& & (446.928) & & (503.696) & & (337.460) \\
 Treat - R3 & & 1370.344$^{***}$ & & 565.438$^{}$ & & 998.665$^{***}$ \\
& & (412.602) & & (403.307) & & (291.103) \\
 P-Val Treat & 0.302 &  & 0.005 &  & 0.007 &  \\
 P-Val Treat FWER & 0.378 &  & 0.009 &  & 0.011 &  \\
 P-Val Treat - R1 &  & <0.001 &  & 0.97 &  

### Table 4

In [21]:
print(latex_tables[2])


\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable: Log Total HH Consumption Trim}} \
\cr \cline{2-7}
\\[-1.8ex] & \multicolumn{2}{c}{Y1} & \multicolumn{2}{c}{Y2} & \multicolumn{2}{c}{Pooled}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) 
 \\ & Overall & By rd & Overall & By rd & Overall & By rd \\
\hline \\[-1.8ex]
 Treat & 0.012$^{}$ & & 0.064$^{*}$ & & 0.036$^{}$ & \\
& (0.030) & & (0.036) & & (0.023) & \\
 Treat - R1 & & -0.033$^{}$ & & 0.064$^{}$ & & 0.013$^{}$ \\
& & (0.047) & & (0.047) & & (0.033) \\
 Treat - R2 & & 0.028$^{}$ & & 0.076$^{*}$ & & 0.049$^{*}$ \\
& & (0.039) & & (0.043) & & (0.029) \\
 Treat - R3 & & 0.038$^{}$ & & 0.052$^{}$ & & 0.044$^{}$ \\
& & (0.042) & & (0.047) & & (0.031) \\
 P-Val Treat & 0.683 &  & 0.08 &  & 0.126 &  \\
 P-Val Treat FWER & 0.683 &  & 0.1 &  & 0.126 &  \\
 P-Val Treat - R1 &  & 0.486 &  & 0.171 &  & 0.687 \\
 P-Val Treat - R1 FWER &  & 0.486 &  & 0.256 &  & 

### Table 5

In [22]:
tables = [results['netsales']['overall'], results['netsales']['by_round'],results['treatMS1MS2_purchase']['overall'], results['treatMS1MS2_sales']['overall']]

stargazer = Stargazer(tables)
stargazer.custom_columns(['Net Sales', 'Effective Price'], [2, 2])
stargazer.rename_covariates({'z': 'Treat','treatMS1MS2_1':'Treat - R1', 'treatMS1MS2_2':'Treat - R2', 'treatMS1MS2_3':'Treat - R3'})
stargazer.significant_digits(3)
stargazer.covariate_order(['z', 'treatMS1MS2_1', 'treatMS1MS2_2', 'treatMS1MS2_3'])

# adding p-values
eff_p_val = ['','']+pval.loc[['purchase','sales'],'treatMS1MS2'].tolist()
eff_p_val_fwer = ['','']+pval_fwer.loc[['purchase','sales'],'treatMS1MS2_fwer'].tolist()

stargazer.add_line('P-Val Treat', ['','']+pval.loc[['purchase','sales'],'treatMS1MS2'].tolist())
stargazer.add_line('P-Val Treat FWER', ['','']+pval_fwer.loc[['purchase','sales'],'treatMS1MS2_fwer'].tolist())

latex_table5 = stargazer.render_latex()

# general formatting
latex_table5 = latex_table5.replace("Adjusted $R^2$", "% Adjusted $R^2$")
latex_table5 = latex_table5.replace("Residual Std. Error", "% Residual Std. Error")
latex_table5 = latex_table5.replace("F Statistic", "% F Statistic")
latex_table5 = latex_table5.replace("\\textit{Note","% \\textit{Note")
latex_table5 = latex_table5.replace("nan","")
latex_table5 = latex_table5.replace("\\begin{table}[!htbp] \\centering", "")
latex_table5 = latex_table5.replace("\\end{table}", "")

# renaming variables
latex_table5 = latex_table5.replace("\\[-1.8ex] & (1) & (2) & (3) & (4) \\",
                                "\\[-1.8ex] & (1) & (2) & (3) & (4) \n \\\ & Overall & By rd & Purchase & Sales \\")


print(latex_table5)


\begin{tabular}{@{\extracolsep{5pt}}lcccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & \multicolumn{2}{c}{Net Sales} & \multicolumn{2}{c}{Effective Price}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) 
 \\ & Overall & By rd & Purchase & Sales \\
\hline \\[-1.8ex]
 Treat & 0.186$^{***}$ & & -57.449$^{**}$ & 145.509$^{***}$ \\
& (0.063) & & (27.156) & (41.767) \\
 Treat - R1 & & -0.205$^{**}$ & & \\
& & (0.096) & & \\
 Treat - R2 & & 0.384$^{***}$ & & \\
& & (0.104) & & \\
 Treat - R3 & & 0.369$^{***}$ & & \\
& & (0.092) & & \\
 P-Val Treat &  &  & 0.034 & <0.001 \\
 P-Val Treat FWER &  &  & 0.043 & 0.001 \\
\hline \\[-1.8ex]
 Observations & 6740 & 6740 & 2014 & 1428 \\
 $R^2$ & 0.098 & 0.102 & 0.089 & 0.066 \\
 % Adjusted $R^2$ & 0.089 & 0.093 & 0.060 & 0.024 \\
 % Residual Std. Error & 2.020 (df=6672) & 2.016 (df=6670) & 639.432 (df=1951) & 789.099 (df=1365) \\
 % F Statistic & 20.619$^{***}$ (df=67; 6672) & 370.686$^{***}$ (df=69; 6670) & 34.249$^{***}$ (df=62; 1951) & 13.002$^{***}$ (d

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 68, but rank is 66
  warnings.warn('covariance of constraints does not have full '
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 70, but rank is 68
  warnings.warn('covariance of constraints does not have full '
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 62, but rank is 61
  warnings.warn('covariance of constraints does not have full '
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 62, but rank is 61
  warnings.warn('covariance of constraints does not 

## Table 6

In [23]:
cleanpricedata_y1y2_tab6 = cleanpricedata_y1y2.copy(deep=True)
cleanpricedata_y1y2_tab6 = cleanpricedata_y1y2_tab6[['salesPrice_trim','hi_1km_wt','hi_3km_wt','hi_5km_wt','monthnum','subloc_1km_wt_grp','subloc_3km_wt_grp','subloc_5km_wt_grp', 'in_sample','MS','lean']]
cleanpricedata_y1y2_tab6['hi'] = pd.NA
cleanpricedata_y1y2_tab6['interact'] = pd.NA
cleanpricedata_y1y2_tab6['interact_lean'] = pd.NA

In [24]:
results = {}
for dist in ['1km_wt', '3km_wt', '5km_wt']:
    df = cleanpricedata_y1y2_tab6.copy(deep=True)
    df.dropna(subset=[f'hi_{dist}','salesPrice_trim','monthnum'], inplace=True)
    mean_price = df[(df['monthnum'] == 0) & (df[f'hi_{dist}'] == 0)]['salesPrice_trim'].mean()
    norm = 100 / mean_price

    # normalize price
    df['salesPrice_trim_norm'] = df['salesPrice_trim'] * norm

    # create hi variable
    df['hi'] = df[f'hi_{dist}']
    df['interact'] = df['monthnum'] * df['hi']

    # regression
    formula = 'salesPrice_trim_norm ~ hi + monthnum + interact'

    for ms in [1,2,3]: # 3 is pooled
        if ms == 3:
            df_filt = df[(df['in_sample'] == 1)]
        else:
            df_filt = df[(df['MS'] == ms) & (df['in_sample'] == 1)]
        model = smf.ols(formula=formula, data=df_filt).fit(cov_type='cluster', cov_kwds={'groups': df_filt[f'subloc_{dist}_grp']})
        results[(dist, ms)] = model

In [25]:
pvals = pd.DataFrame()
# storeing pval in a df
for dv in ['hi', 'monthnum', 'interact']:
    val = {(k[0], k[1]): np.round(v.pvalues[dv],3) for k, v in results.items()}
    pvals[dv] = pd.Series(val)

# keep only columns 3km_wt and 3rd column in 1km_wt and 5km_wt
pvals = pvals.T
pvals = pvals[[('3km_wt', 1), ('3km_wt', 2), ('3km_wt', 3), ('1km_wt', 3), ('5km_wt', 3)]]

In [ ]:
def cgmwildboot(data, formula, n_bootstraps, cluster, bootcluster)
    

In [26]:
def wild_bootstrap(data, model, n_bootstraps, dv, clust_var):
    """
    Wild Cluster Bootstrap-t with random signs within clusters
    """
    cluster_var = data[clust_var]
    unique_clusters = cluster_var.unique()
    boot_results = []

    for _ in range(n_bootstraps):
        boot_data = data.copy()
        # resample residuals within each cluster
        for cluster in unique_clusters:
            cluster_indices = data[cluster_var == cluster].index

            # multiply residuals by random signs, either -1 or 1, (radmacher weights)
            signs = np.random.choice([-1, 1], size=len(cluster_indices))
            boot_data.loc[cluster_indices, dv] = model.predict(data.loc[cluster_indices]) + signs * model.resid.loc[cluster_indices]

        # Refit model on bootstrapped data
        boot_model = smf.ols(model.model.formula, data=boot_data).fit()
        boot_results.append(boot_model.params)

    return np.array(boot_results)[:,1:4]


In [27]:
bootstrap_res = {}
bootstrap_pvals = pd.DataFrame(index=pd.MultiIndex.from_product([['1km_wt', '3km_wt', '5km_wt'], [1, 2, 3]], names=['dist', 'ms']), columns=['hi', 'monthnum', 'interact'])
n_bootstraps = 9

for dist  in ['1km_wt', '3km_wt', '5km_wt']:
    df = cleanpricedata_y1y2_tab6.copy(deep=True)
    df.dropna(subset=[f'hi_{dist}','salesPrice_trim','monthnum'], inplace=True)
    mean_price = df[(df['monthnum'] == 0) & (df[f'hi_{dist}'] == 0)]['salesPrice_trim'].mean()
    norm = 100 / mean_price

    # normalize price
    df['salesPrice_trim_norm'] = df['salesPrice_trim'] * norm
    df['salesPrice_trim_norm'] = df['salesPrice_trim_norm'].astype(float)

    # create hi variable
    df['hi'] = df[f'hi_{dist}']
    df['interact'] = df['monthnum'] * df['hi']

    # regression
    formula = 'salesPrice_trim_norm ~ hi + monthnum + interact'


    for ms in [1,2,3]: # 3 is pooled
        if ms == 3:
            df_filt = df[(df['in_sample'] == 1)]
        else:
            df_filt = df[(df['MS'] == ms) & (df['in_sample'] == 1)]
        res = wild_bootstrap(df_filt, results[(dist, ms)], n_bootstraps, 'salesPrice_trim_norm', f'subloc_{dist}_grp')
        bootstrap_res[(dist,ms)] = res

        model = results[(dist, ms)]

        for i, var in enumerate(['hi', 'monthnum', 'interact']):
            observed_coef = model.params[var]
            # calculating p-values as proportion of bootstrap coefs where abs(boot_coef) >= abs(obs_coef)
            p_value = np.round(np.mean(np.abs(bootstrap_res[(dist,ms)][:,i]) >= np.abs(observed_coef)),3)

            # store p-value in df
            bootstrap_pvals.loc[(dist,ms),var] = p_value


In [28]:
bootstrap_pvals = bootstrap_pvals.T
bootstrap_pvals = bootstrap_pvals[[('3km_wt', 1), ('3km_wt', 2), ('3km_wt', 3), ('1km_wt', 3), ('5km_wt', 3)]]

In [29]:
# use stargazer to create a table
result_list = [results[('3km_wt', 1)], results[('3km_wt', 2)], results[('3km_wt', 3)], results[('1km_wt', 3)], results[('5km_wt', 3)]]
stargazer = Stargazer(result_list)

# configure Stargazer object for output
stargazer.custom_columns(['Main Specification (3km)', 'Robustness (Pooled)'], [3, 2])
stargazer.rename_covariates({'hi': 'High', 'monthnum': 'Month', 'interact': 'High x Month'})
stargazer.show_degrees_of_freedom(False)
stargazer.significant_digits(3)
stargazer.covariate_order(['hi', 'monthnum', 'interact'])
# add p-values as a rows
stargazer.add_line('P-value High', pvals.loc['hi'].values.tolist())
stargazer.add_line('P-value Treat Bootstrap', bootstrap_pvals.loc['hi'].values.tolist())
stargazer.add_line('P-value Month', pvals.loc['monthnum'].values.tolist())
stargazer.add_line('P-value High Bootstrap', bootstrap_pvals.loc['monthnum'].values.tolist())
stargazer.add_line('P-value High x Month', pvals.loc['interact'].values.tolist())
stargazer.add_line('P-value Treat x High Bootstrap', bootstrap_pvals.loc['interact'].values.tolist())


latex_table6 = stargazer.render_latex()

# edit the latex tables
latex_table6 = latex_table6.replace("\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\",
                                "\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \n \\\ & Y1 & Y2 & Pooled & 1km & 5km \\")
latex_table6 = latex_table6.replace("Adjusted $R^2$", "% Adjusted $R^2$")
latex_table6 = latex_table6.replace("Residual Std. Error", "% Residual Std. Error")
latex_table6 = latex_table6.replace("F Statistic", "% F Statistic")
latex_table6 = latex_table6.replace("\\textit{Note","% \\textit{Note")
latex_table6 = latex_table6.replace("salesPrice_trim_norm","Trimmed Sales Price")
latex_table6 = latex_table6.replace("\\begin{table}[!htbp] \\centering", "")
latex_table6 = latex_table6.replace("\\end{table}", "")

print(latex_table6)


\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{5}{c}{\textit{Dependent variable: Trimmed Sales Price}} \
\cr \cline{2-6}
\\[-1.8ex] & \multicolumn{3}{c}{Main Specification (3km)} & \multicolumn{2}{c}{Robustness (Pooled)}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) 
 \\ & Y1 & Y2 & Pooled & 1km & 5km \\
\hline \\[-1.8ex]
 High & 4.410$^{**}$ & 2.855$^{}$ & 3.970$^{**}$ & 2.787$^{}$ & 3.766$^{**}$ \\
& (2.091) & (1.992) & (1.817) & (1.719) & (1.822) \\
 Month & 1.189$^{***}$ & 1.224$^{***}$ & 1.364$^{***}$ & 1.327$^{***}$ & 1.537$^{***}$ \\
& (0.363) & (0.377) & (0.350) & (0.339) & (0.291) \\
 High x Month & -0.574$^{}$ & -0.476$^{}$ & -0.573$^{}$ & -0.520$^{}$ & -0.835$^{**}$ \\
& (0.422) & (0.459) & (0.386) & (0.390) & (0.366) \\
 P-value High & 0.035 & 0.152 & 0.029 & 0.105 & 0.039 \\
 P-value Treat Bootstrap & 0.667 & 0.444 & 0.444 & 0.556 & 0.333 \\
 P-value Month & 0.001 & 0.001 & 0.0 & 0.0 & 0.0 \\
 P-value High Bootstrap & 0.556 & 

## Table 7

In [30]:
# copy the raw data and create columns for treatment and interaction variable
ms1ms2_pooled_tab7 = ms1ms2_pooled.copy(deep=True)
# filter relevant columns
ms1ms2_pooled_tab7 = ms1ms2_pooled_tab7[['oafid', # id
                                         'treat12', 'treat13', 'treatMS1MS2', # treatment variables
                                         'inventory_trim', 'netrevenue_trim', 'logtotcons_trim', # outcome variables
                                         'Y1round2', 'Y1round3', 'Y2round1', 'Y2round2', 'Y2round3','hi','subloc','interviewdate']] # independent variables

ms1ms2_pooled_tab7.sort_index(inplace=True)
ms1ms2_pooled_tab7['z'] = pd.NA
ms1ms2_pooled_tab7['z_hi'] = pd.NA

### Running the first set of regressions

In [31]:
# list of treaments
treatments = ['treat12', 'treat13', 'treatMS1MS2']

# list of dependent variables
dependent_vars = ['inventory_trim', 'netrevenue_trim', 'logtotcons_trim']

# list of changeing independent variables depending on the treatment
independent_vars = {
    'treat12': 'Y1round2 + Y1round3',
    'treat13': 'Y2round2 + Y2round3',
    'treatMS1MS2': 'Y1round2 + Y1round3 + Y2round1 + Y2round2 + Y2round3'
    }

# empty dictionary to store results
results = {}
pvals = {var: [] for var in ['z', 'hi', 'z_hi','z+z_hi']}

# Simulating the loop to replace variables and run regressions
for dv in dependent_vars:
    for treat in treatments:
        # Stata automatically omits the missing values in the regression – here we have to do it manually so we copy the data and drop variables
        df = ms1ms2_pooled_tab7.copy(deep=True)
        df = df.dropna(subset=[dv, treat, 'hi', 'subloc','interviewdate'])
        # setting treament variable
        df['z'] = df[treat] # setting z to the treatment variable

        # setting interaction variable
        df['z_hi'] = df[treat]*df['hi'] # setting z_hi to the interaction of the treatment hi saturation

        # setting the formula to run the regression
        formula = f'{dv} ~ z + hi + z_hi + interviewdate + {independent_vars[treat]}'

        # Run the regression
        model_key = f'model_{dependent_vars.index(dv)*len(treatments) + treatments.index(treat)}'
        results[model_key] = smf.ols(formula, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['subloc']})

        # test the hypothesis that z + z_hi = 0
        hypothesis = 'z + z_hi = 0'
        t_test = results[model_key].t_test(hypothesis)

        # store p-value round to 3 decimals
        pvals['z+z_hi'].append(np.round(t_test.pvalue,3))
        pvals['z'].append(np.round(results[model_key].pvalues['z'],3))
        pvals['hi'].append(np.round(results[model_key].pvalues['hi'],3))
        pvals['z_hi'].append(np.round(results[model_key].pvalues['z_hi'],3))

### Running boostrap regressions

In [32]:
n_bootstraps = 999  # reported data is based on 999 iterations
bootstrap_res = {}
bootstrap_pvals = {var: [] for var in ['z', 'hi', 'z_hi']}

for dv in dependent_vars:
    for treat in treatments:
        df = ms1ms2_pooled_tab7.copy(deep=True)
        df = df.dropna(subset=[dv, treat, 'hi', 'subloc', 'interviewdate'])
        df['z'] = df[treat]
        df['z_hi'] = df[treat] * df['hi']
        df[dv] = df[dv].astype(float)

        formula = f'{dv} ~ z + hi + z_hi + interviewdate + {independent_vars[treat]}'
        model_key = f'model_{dependent_vars.index(dv)*len(treatments) + treatments.index(treat)}'
        model = results[model_key]

        # Wild bootstrap
        res = wild_bootstrap(df, model, n_bootstraps, dv,'subloc')
        bootstrap_res[model_key] = res

        for i, var in enumerate(['z', 'hi', 'z_hi']):
            observed_coef = model.params[var]
            # calculating p-values as proportion of bootstrap coefs where abs(boot_coef) >= abs(obs_coef)
            p_value = np.round(np.mean(np.abs(bootstrap_res[model_key][:,i]) >= np.abs(observed_coef)),3)
            bootstrap_pvals[var].append(p_value)

### Output to LaTeX

In [33]:
# use stargazer to create a table
result_list = list(results.values())
stargazer = Stargazer(result_list)

# configure Stargazer object for output
stargazer.custom_columns(['Inventory', 'Net Revenues', 'Consumption'], [3, 3, 3])
stargazer.rename_covariates({'z': 'Treat', 'hi': 'High', 'z_hi': 'Treat x High'})
stargazer.show_degrees_of_freedom(False)
stargazer.significant_digits(3)
stargazer.covariate_order(['z', 'hi', 'z_hi'])
# add p-values as a rows
stargazer.add_line('P-value T + TH = 0', pvals['z+z_hi'])
stargazer.add_line('P-value Treat', pvals['z'])
stargazer.add_line('P-value Treat Bootstrap', bootstrap_pvals['z'])
stargazer.add_line('P-value High', pvals['hi'])
stargazer.add_line('P-value High Bootstrap', bootstrap_pvals['hi'])
stargazer.add_line('P-value Treat*High', pvals['z_hi'])
stargazer.add_line('P-value Treat*High Bootstrap', bootstrap_pvals['z_hi'])


latex_table7 = stargazer.render_latex()

# edit the latex table to add row for telling if Y1 Y2 or Pooled after \\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \\
latex_table7 = latex_table7.replace("\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \\",
                                "\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \n \\\ & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled \\")
latex_table7 = latex_table7.replace("Adjusted $R^2$", "% Adjusted $R^2$")
latex_table7 = latex_table7.replace("Residual Std. Error", "% Residual Std. Error")
latex_table7 = latex_table7.replace("F Statistic", "% F Statistic")
latex_table7 = latex_table7.replace("\\textit{","% \\textit{")
latex_table7 = latex_table7.replace("\\begin{table}[!htbp] \\centering", "")
latex_table7 = latex_table7.replace("\\end{table}", "")

print(latex_table7)


\begin{tabular}{@{\extracolsep{5pt}}lccccccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & \multicolumn{3}{c}{Inventory} & \multicolumn{3}{c}{Net Revenues} & \multicolumn{3}{c}{Consumption}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) 
 \\ & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled \\
\hline \\[-1.8ex]
 Treat & 0.759$^{***}$ & 0.546$^{***}$ & 0.740$^{***}$ & 1059.602$^{**}$ & 1193.768$^{*}$ & 1101.389$^{**}$ & 0.012$^{}$ & -0.051$^{}$ & -0.011$^{}$ \\
& (0.189) & (0.185) & (0.155) & (437.732) & (685.048) & (430.091) & (0.040) & (0.040) & (0.023) \\
 High & 0.124$^{}$ & -0.028$^{}$ & 0.017$^{}$ & 533.903$^{}$ & -152.603$^{}$ & 164.936$^{}$ & -0.003$^{}$ & -0.084$^{}$ & -0.047$^{}$ \\
& (0.355) & (0.219) & (0.241) & (551.179) & (558.948) & (479.685) & (0.051) & (0.053) & (0.043) \\
 Treat x High & -0.333$^{}$ & -0.065$^{}$ & -0.291$^{}$ & -1114.628$^{**}$ & -555.215$^{}$ & -816.770$^{}$ & -0.013$^{}$ & 0.174$^{***}$ & 0.067$^{*}$ \\
& (0.229) &

## Table 8

In [34]:
tab8_dt = ms1ms2_pooled.loc[:, ['treatMS1MS2', 'hi', 'treatMS1MS2hi', 'interviewdate', 'netrevenue_trim', 'strata_group', 'groupnum', 'subloc', 'Y1round1', 'Y1round2', 'Y1round3', 'Y2round1', 'Y2round2', 'Y2round3']].dropna()
tab8_dt['net_revenue_3'] = tab8_dt['netrevenue_trim'] * 3
model = smf.ols('net_revenue_3 ~ treatMS1MS2 + hi + treatMS1MS2hi + interviewdate + Y1round1 + Y1round2 + Y1round3 + Y2round1 + Y2round2 + Y2round3', data=tab8_dt)
results_t8 = model.fit(cov_type='cluster', cov_kwds={'groups': tab8_dt['subloc']})
model_params_t8 = results_t8.params

In [35]:
# Annualized coefficients
treat_coef = model_params_t8['treatMS1MS2']
treat_hi_coef = (model_params_t8['treatMS1MS2'] + model_params_t8['treatMS1MS2hi'])
hi_coef = model_params_t8['hi']

# Direct beneficiary population
direct_beneficiary_pop_low = 247.0
direct_beneficiary_pop_high = 495.0

# Total direct gains
total_direct_gains_low = treat_coef * direct_beneficiary_pop_low
total_direct_gains_high = treat_hi_coef * direct_beneficiary_pop_high

# Total indirect gains (only applicable to high saturation areas)
total_indirect_gains_high = hi_coef * 3553.0

# Total gains
total_gains_low = total_direct_gains_low
total_gains_high = total_direct_gains_high + total_indirect_gains_high

# Fraction of gains direct
fraction_gains_direct_low = 1  # All gains are direct in low saturation
fraction_gains_direct_high = total_direct_gains_high / total_gains_high

# Fraction of gains indirect (only applicable to high saturation areas)
fraction_gains_indirect_high = total_indirect_gains_high / total_gains_high

table_8 = {
    "1. Direct gains/HH (Ksh)": [treat_coef, treat_hi_coef],
    "2. Indirect gains/HH (Ksh)": [0, hi_coef],
    "3. Ratio of indirect to direct gains": [0, hi_coef / treat_hi_coef],
    "4. Direct beneficiary population (HH)": [direct_beneficiary_pop_low, direct_beneficiary_pop_high],
    "5. Total local population (HH)": [3553.0, 3553.0],
    "6. Total direct gains (Ksh)": [total_direct_gains_low, total_direct_gains_high],
    "7. Total indirect gains (Ksh)": [0, total_indirect_gains_high],
    "8. Total gains (direct + indirect; Ksh)": [total_gains_low, total_gains_high],
    "9. Fraction of gains direct": [fraction_gains_direct_low, fraction_gains_direct_high],
    "10. Fraction of gains indirect": [0, fraction_gains_indirect_high],
}

# Convert the calculations to DataFrame and transpose it
table_8_df = pd.DataFrame(table_8, index=["Low Saturation", "High Saturation"]).T

# Now you can print table_8_df to see the recreated table
table_8_df

latex_table8 = table_8_df.to_latex(index=True, float_format="%.3f")
print(latex_table8)

\begin{tabular}{lrr}
\toprule
 & Low Saturation & High Saturation \\
\midrule
1. Direct gains/HH (Ksh) & 3304.166 & 853.856 \\
2. Indirect gains/HH (Ksh) & 0.000 & 494.807 \\
3. Ratio of indirect to direct gains & 0.000 & 0.579 \\
4. Direct beneficiary population (HH) & 247.000 & 495.000 \\
5. Total local population (HH) & 3553.000 & 3553.000 \\
6. Total direct gains (Ksh) & 816128.985 & 422658.745 \\
7. Total indirect gains (Ksh) & 0.000 & 1758050.851 \\
8. Total gains (direct + indirect; Ksh) & 816128.985 & 2180709.596 \\
9. Fraction of gains direct & 1.000 & 0.194 \\
10. Fraction of gains indirect & 0.000 & 0.806 \\
\bottomrule
\end{tabular}

